In [1]:
#to create a soup objet where to search
from bs4 import BeautifulSoup as bs
#to read the page
import requests
# we need splinter 
from splinter import Browser
# to create sleeps we need
import time
# to scrape tables from web
import pandas as pd

In [2]:
#start splinter in OSX
def init_browser():
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser("chrome", **executable_path, headless = False)
browser=init_browser()


# Scrape NASA's Mars page

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:

#Page to be scraped, the lates news I found, and choosed manualy
nasa = "https://mars.nasa.gov/news/"
#Lets us splinter at once. (When I tried using a request, the html had some problems who didn't throw us the full html)
browser.visit(nasa)
time.sleep(2)
html = browser.html
#Take the html into a soup object
soup = bs(html,"html.parser")

In [4]:
marsnews={}
#Take the first title you find
titulo = soup.find("div",class_="list_text").a.text
#Take the firs article teaser you find
parafo = soup.find("div", class_="article_teaser_body").text
#add them into marsnews dict
marsnews['titulo'] = titulo
marsnews['parafo'] = parafo

In [5]:
marsnews

{'titulo': "The Detective Aboard NASA's Perseverance Rover",
 'parafo': 'An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.'}

# JPL Mars Space Images - Featured Image

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [6]:
nasa_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(nasa_img)
time.sleep(2)
html = browser.html
# new soup
soup = bs(html,"html.parser")

In [7]:
#Search for the link
img_link = soup.find("section", class_="grid_gallery module grid_view").ul.li.a["data-fancybox-href"]
print(img_link)
#The given link was incomplete so... 
#I found this requests function that allows to join an absolute path to the relative path within 
img_link = requests.compat.urljoin(nasa_img, img_link)
#So here we have the complete link
img_link

/spaceimages/images/largesize/PIA23932_hires.jpg


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23932_hires.jpg'

# Mars Weather

Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather

In [32]:
#New browser visit
nasa_twitt = "https://twitter.com/marswxreport?lang=en"
browser.visit(nasa_twitt)
time.sleep(2)
html = browser.html
#New soup
soup = bs(html, "html.parser")

In [59]:
#The page has the problem that sometimes appears foreign tweets, that doesn't have de info we want
#So first we have to take all the tweets
tweets = soup.main.section.div.find_all("div", {"data-testid" : "tweet"})

#Then we iterate through tweets 
for tweet in tweets:
    # Make sure the tweet is from Mars Weather
    author = tweet.span.string
    if author == "Mars Weather":
        #If it is, take the text of the tweet
        mars_weather = tweet.find("div", {"lang":"en"}).span.string.strip()
        #Stop iterate
        break
mars_weather
 


'InSight sol 534 (2020-05-27) low -92.8ºC (-135.1ºF) high 0.9ºC (33.7ºF)\nwinds from the SW at 4.8 m/s (10.8 mph) gusting to 18.8 m/s (42.1 mph)\npressure at 7.20 hPa'

# Mars Facts

Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

In [15]:
facts_page = "https://space-facts.com/mars/"
browser.visit(facts_page)
time.sleep(2)
html = browser.html
#Soup
soup = bs(html, "html.parser")

In [11]:
#to make posible to pandas to read a soup element i had to transform the string into a prettify
table_loc = soup.section.article.find("div", id="text-75").table.prettify()
#now we can use "read_html" from pandas
table = pd.read_html(table_loc)
#But to use it for our next page we have to convert it again into html
#First I transform to a DataFrame
table = pd.DataFrame(table[0])
#And then to a html, without index and header
table = table.to_html(index=False, header=False )


# Mars Hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

In [30]:
hemispage = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispage)
time.sleep(2)
html = browser.html
#Soup
soup = bs(html, "html.parser")

In [31]:
hemisphere_image_url = [] 
#First we get in a list the four parts with the different hemispheres 
hemispheres = soup.section.find("div", class_="collapsible results").find_all("div", class_="item")

#Then we iterate through the 2 hemispheres
for x in hemispheres:
    #Take the href link to the page who has the image and complete it
    link = x.a["href"]
    link = requests.compat.urljoin(hemispage, link)
    #Visit the new page
    browser.visit(link)
    time.sleep(2)
    # Read the html of the new page
    html = browser.html
    soup = bs(html, "html.parser")
    #Get the title and the links to image and put them in a dictionary
    title = soup.section.h2.string.replace("Enhanced", "")
    links = soup.div.ul.li.a["href"]
    dictionary = {"title" : title , "img_link" : links }
    #apend the dictionary to the list of urls
    hemisphere_image_url.append(dictionary)
    #Get back to the original page
    browser.back()
#check ifall right
print(hemisphere_image_url)

[{'title': 'Cerberus Hemisphere ', 'img_link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
